# TFG Teoría de Grafos y Criptografía

Autor: Carlota Valdivia Manzano


# Algoritmo de Cifrado basado en Teoría de Grafos

El algoritmo propuesto por [*Wael Etaiwi*](https://www.researchgate.net/publication/269803082_Encryption_Algorithm_Using_Graph_Theory?enrichId=rgreq-3bcb3d1a96a5b95b5a828a7dcc7e36f6-XXX&enrichSource=Y292ZXJQYWdlOzI2OTgwMzA4MjtBUzozMzgwMzU1NjU3MTEzNjJAMTQ1NzYwNTM2NzM0NA%3D%3D&el=1_x_2&_esc=publicationCoverPdf)  es un algoritmo de cifrado simétrico que hace uso de los conceptos de grafo completo, ciclo y árbol generador minimal para poder generar cifrados usando una clave compartida.

En este cuaderno, presentaremos una implementación del algoritmo y un ejemplo de cifrado y descifrado haciendo uso del mismo.

## Implementación

Para la implementación de los algoritmos ha sido conveniente crear una clase Grafo, una tabla de codificación y algunos métodos auxiliares.

### Clase Grafo

La clase Grafo está compuesta por dos atributos:
* *V*: conjunto de vértices
* *Matrix*: matriz de adyacencia del grafo,

y por un constructor y tres métodos:
* Constructor con parámetros
* *primMST*: Método para calcular un árbol generador minimal (MST) usando el algoritmo de Prim
* *printMST*: Método para imprimir el MST
* *minWeight*: Método para encontrar el vértice con menor distancia a cualquier otro vértice no procesado en el MST

In [7]:
import sys # Lo importamos para usar el valor INT_MAX
import math 
import numpy as np 
import time

In [8]:
class Graph():

    def __init__(self, V):
        """
            'V' es el número de vértices que posee el grafo 
            'self.matrix' contendrá la matriz de adyacencia del grafo
        """
        self.V = V
        self.matrix = [[0 for column in range(V)] for row in range(V)]

 
    # Método para imprimir el MST
    def printMST(self, parent):
        print("Arista \tPeso")
        for i in range(1, self.V):
            print(parent[i], "-", i, "\t", self.matrix[i][parent[i]])
 

    # Método para encontrar el vértice con menor distancia al
    # cualquier otro vértice que no esté ya procesado en el árbol
    def minWeight(self, weight, mstSet):
        """
            'weight' es el peso a comprobar
            'mstSet' es el conjunto de vértices que están procesados 
                     en el árbol
        """ 
 
        # Inicializamos el valor mínimo
        min = sys.maxsize
 
        # Recorremos el conjunto de vértices
        for v in range(self.V):
            # Si el peso es menor que el mínimo y no está en el árbol
            # actualizamos los valores del mínimo
            if weight[v] < min and mstSet[v] == False:
                min = weight[v]
                min_index = v
                        
        return min_index
 
    # Método para construir un Árbol Generador Minimal (MST) con el 
    # algoritmo de Prim con colas de prioridad
    def primMST(self):
 
        # Lista de pesos inicializadas con valor infinito
        weight = [sys.maxsize] * self.V
        # Lista para almacenar el MST
        parent = [None] * self.V  # Array to store constructed MST
        # Asignamos los valores 0 y -1 al vértice raíz
        weight[0] = 0
        parent[0] = -1  
        # Lista para seleccionar los vértices procesados
        mstSet = [False] * self.V
 
        # Realizamos tantas iteraciones como vértices
        for i in range(self.V):
             
            # Elegimos el vértice con menor distancia del conjunto de
            # vértices sin procesar
            v = self.minWeight(weight, mstSet)
 
            # Seleccionamos como procesado el vértice
            mstSet[v] = True

            # Actualizamos las etiquetas de los vértices adyacentes
            for u in range(self.V):
 
                # Si el vértice no ha sido procesado, y su distancia calculada
                # es menor que la guardada en la variable peso actualizamos
                if  mstSet[u] == False and self.matrix[v][u] != 0 and self.matrix[v][u] < weight[u]:
                    weight[u] = self.matrix[v][u]
                    parent[u] = v
                    
        # Imprimimos el árbol generador minimal
        #self.printMST(parent)
        
        # Construimos la matriz de adyacencia del árbol generador minimal
        T = [[0 for column in range(self.V)] for row in range(self.V)]
        
        # Realizamos tantas iteraciones como vértices
        for i in range(self.V):
            T[parent[i]][i] = self.matrix[i][parent[i]]
            T[i][parent[i]] = self.matrix[i][parent[i]]
            
        return T
            

### Tabla de codificación

La **tabla de codificación** es un diccionario donde cada carácter, en este caso letra del abecedario, está asociado a un código. Dicho código es necesario para poder calcular el peso de las aristas.

In [9]:
tabla_codificacion = {
        'A' : '1', 
        'B' : '2', 
        'C' : '3', 
        'D' : '4', 
        'E' : '5', 
        'F' : '6', 
        'G' : '7', 
        'H' : '8', 
        'I' : '9', 
        'J' : '10', 
        'K' : '11', 
        'L' : '12', 
        'M' : '13', 
        'N' : '14', 
        'O' : '15', 
        'P' : '16', 
        'Q' : '17', 
        'R' : '18', 
        'S' : '19', 
        'T' : '20', 
        'U' : '21', 
        'V' : '22', 
        'W' : '23', 
        'X' : '24', 
        'Y' : '25', 
        'Z' : '26'
    }

### Función para obtener el peso de una arista según la codificación

Se ha implementado también una función para poder calcular el peso de una arista a partir de la tabla de codificación anterior.

In [10]:
# Función para obtener el peso de una arista en función
# de los dos caracteres que la componen
def codificarLetras(l1,l2):
    """
        'l1' primera letra a codificar
        'l2' segunda letra a codificar
    """ 
    
    return int(tabla_codificacion[l1])-int(tabla_codificacion[l2])

### Función para obtener la letra correspondiente a un valor

Función para obtener la letra que corresponde a un valor numérico en la tabla de codificación.

In [11]:
# Función para obtener la letra correspondiente a un valor
# de la tabla de codificación
def obtenerLetra(val):
    """
        'val' valor de la letra correspondiente en la tabla de
              codificación
    """ 
    for letra, valor in tabla_codificacion.items():
         if float(val) == float(valor):
            return letra
        
    return "No existe esa letra"

### Función para obtener la codificación según el peso de una arista

Se ha implementado también una función para poder calcular el peso de una arista a partir de la tabla de codificación anterior.

In [12]:
# Función para obtener la codificación de un carácter según
# el peso de una arista
def decodificarLetra(peso,l):
    """
        'peso' peso de la arista
        'l' letra del otro extremo de la arista
    """     
    return obtenerLetra(str(peso+int(tabla_codificacion[l])))

### Función para obtener un Grafo formado por los caracteres

Esta función obtiene un grafo con su respectiva representación matricial a partir del mensaje que quieres cifrar y de un carácter especial para denotar por donde empieza el mensaje.

In [13]:
# Función que dado un mensaje un carácter especial nos
# devuelve un grafo que representa dicho mensaje
def grafoMensaje(mensaje, carac):
    """
        'mensaje' texto plano al que convertir en grafo
        'carac' carácter especial para indicar el carácter 
                inicial del mensaje
    """ 
    
    # Almacenamos el número de vértices de nuestro grafo
    n = len(mensaje)+1  # Sumamos uno por el caracter especial que le añadimos
    
    # Creamos un grafo de tamaño n
    grafo = Graph(n)
    
    # Creamos una lista con el peso de las aristas
    if grafo.V < 3:
        pesos = [0] * (grafo.V-1)
    else:
        pesos = [0] * grafo.V
    
    # Calculamos el peso de las aristas a partir de la codificación    
    for u in range(len(pesos)):
        if u == 0:
            pesos[u] = codificarLetras(mensaje[u],carac)
        else:
            pesos[u] = codificarLetras(mensaje[u%(n-1)],mensaje[u-1])
        
        
    # Agregamos un contador para los pesos de las aristas a añadir
    cont = 0
    # Seleccionamos el peso máximo
    max_peso = int(tabla_codificacion['Z'])
    # Creamos una lista con los pesos a añadir
    pesos_c = [0] * int((n-1)*(n-3)/2)    
    if len(pesos_c) != 0:
        for u in range(len(pesos_c)):
            pesos_c[u] = max_peso + 1
            max_peso = max_peso + 1
    
                    
    # Añadimos los pesos a la matriz de adyacencia
    for u in range(grafo.V):
        # Caso del vértice añadido para señalar el comienzo
        if u == 0:
            grafo.matrix[u][u+1] = pesos[u]
            grafo.matrix[u+1][u] = pesos[u]
        else:
            # Como la matriz es simétrica recorremos solamente
            # la diagonal superior
            for v in range(u+1,grafo.V):
                # Caso de vértice adyacente siguiente
                if (u+1)%(n-1) == v%(n-1):
                    grafo.matrix[u][v] = pesos[u]
                    grafo.matrix[v][u] = pesos[u]
                # Caso de vértice adyacente anterior
                elif (v+1)%(n-1) == u%(n-1):
                    grafo.matrix[u][v] = pesos[v]
                    grafo.matrix[v][u] = pesos[v]
                # Caso de aristas añadidas para hacer 
                # el grafo completo
                else:
                    grafo.matrix[u][v] = pesos_c[cont]
                    grafo.matrix[v][u] = pesos_c[cont]
                    cont = cont + 1

    return grafo
    

### Función para sustituir la diagonal por el orden de los caracteres

In [14]:
# Función que dada una matriz sustituye su diagonal principal por
# el orden de los caracteres en el grafo
def sustituirDiag(T):
    """
        'T' matriz a la que sustituir la diagonal principal
    """ 
    for u in range(len(T)):
        T[u][u] = u

### Función para convertir las matrices cifradas en formato lineal 

Función que dada una matriz, obtendremos una lista con sus entradas en formato lineal ordenadas por filas.

In [15]:
# Función que dada una matriz obtiene una única lista con
# todos sus entradas ordenadas por filas
def matrizLineal(M):
    """
        'M' matriz de la que obtener una lista con sus entradas
    """ 
    L = []
    
    # Recorremos las entradas de la matriz por filas
    # y los añadimos a la lista
    for fila in M:
        for i in fila:
            L.append(i)
    return L

### Función para convertir formato lineal en matrices

Función que dada una lista con entradas, obtenemos la matriz cuadrada cuyos elementos corresponden con las entradas de la lista ordenados por filas.

In [16]:
# Función que dada una lista de componentes de tamaño
# cuadrado obtiene una matriz con sus entradas 
# ordenadas por filas
def linealMatriz(l):
    """
        'l' lista de la que obtener una matriz con sus entradas
    """ 
    n = int(math.sqrt(len(l)))
    M = [[0 for column in range(n)] for row in range(n)]
    
    # Recorremos los elementos de la lista
    for i in range(n):
        for j in range(n):
            M[i][j] = l[i*n+j]
    return M

### Función para descrifrar un mensaje de un MST

In [17]:
# Función que dado un árbol generador minimal y un 
# carácter inicial descriframos un mensaje a usando
# una tabla de codificación
def descifrarMatriz(T,carac):
    """
        'T' árbol generador minimal de 
        'carac' carácter especial para indicar el carácter 
                inicial del mensaje
    """ 
    # Número de letras de la palabra a descifrar
    V = len(T)-1
    # Lista para almacenar la palabra a descifrar
    mensaje = [None] * V
    # Carácter inicial con el que empezar a descifrar
    letra = carac
    # Número de letras descifradas
    tam = 1
    # Contador de iteraciones
    i = 1
    
    # Añadimos la primera letra
    mensaje[0]=(decodificarLetra(T[0][1],letra))
    
    # Recorremos las entradas de la matriz por filas
    # y los añadimos a la lista
    while tam < V:
        # Comenzamos iterando a partir de i
        cont = i+1
        
        # Mientras que queden letras por descifrar y no hayamos recorrido la fila entera
        while tam < V and cont <= V:
            
            # Comprobamos que el peso no sea 0, y que la letra asociada al nº de fila o columna sea una
            # vacía y la otra no
            if T[i][cont] != 0 and mensaje[cont-1] == None and mensaje[i-1] != None:
                # Si el extremo es el siguiente
                if i+1 == cont:
                    mensaje[cont-1]=(decodificarLetra(T[i][cont],mensaje[i-1]))
                # Si es el anterior
                elif (cont+1)%V == i:
                    mensaje[cont-1]=(decodificarLetra(-(T[i][cont]),mensaje[i-1]))
                # Aumentamos el nº de letras descifradas
                tam = tam + 1
                
            # Comprobamos que el peso no sea 0, y que la letra asociada al nº de fila o columna sea una
            # vacía y la otra no
            elif T[i][cont] != 0 and mensaje[cont-1] != None and mensaje[i-1] == None:
                # Si el extremo es el siguiente
                if i+1 == cont:
                    mensaje[i-1]=(decodificarLetra(-(T[i][cont]),mensaje[cont-1]))
                # Si es el anterior
                elif (cont+1)%V == i:
                    mensaje[i-1]=(decodificarLetra(T[i][cont],mensaje[cont-1]))
                # Aumentamos el nº de letras descifradas
                tam = tam + 1
                                
            # Aumentamos el contador    
            cont = cont + 1
            
        # Aumentamos el contador
        if i == V:
            i = 1
        else:
            i = i + 1

    return mensaje

### Algoritmo de cifrado

In [73]:
# Algoritmo de cifrado dado un mensaje, un carácter inicial y
# una clave compartida
def codificar(mensaje, carac, K):
    """
        'mensaje' texto plano al que convertir en grafo 
        'carac' carácter especial para indicar el carácter inicial del mensaje
        'K' matriz de clave compartida  
    """ 
    # 1. Construimos un grafo g cuyos vértices son los caracteres del mensaje 
    # y el carácter especial
    g = grafoMensaje(mensaje, carac)
    M = g.matrix
    
    # 2. Comprobamos si la matriz M es invertible
    if np.linalg.det(M) != 0:

        # 3. Encontramos la matriz de adyacencia del árbol generador minimal (MST) 
        # del grafo g usando el Algoritmo de Prim
        T = g.primMST()

        # 4. Sustituimos los ceros de la diagonal principal por el orden de los 
        # caracteres
        sustituirDiag(T)

        # 5. Multiplicamos la matriz de adyacencia del grafo, M, y la del árbol 
        # generador minimal, T, para obtener la matriz P
        P = np.dot(M,T).tolist()

        # 6. Usamos la clave K para cifrar P
        C = np.dot(P,K).tolist()

        # 7. Pasamos las matrices C y M a formato lineal 
        lista1 = matrizLineal(C)
        lista2 = matrizLineal(M)
        mensaje_cifrado = lista1 + lista2

        return mensaje_cifrado



### Algoritmo de descifrado

In [19]:
# Algoritmo de descifrado dado un mensaje, un carácter inicial 
# y una clave compartida
def decodificar(mensaje_cifrado, carac, K):
    """
        'mensaje' texto plano al que convertir en grafo 
        'carac' carácter especial para indicar el carácter inicial del mensaje
        'K' matriz de clave compartida  
    """ 
    # 1. Dividimos el mensaje cifrado en dos listas y lo transformarmos en dos 
    # matrices, C_desc y M_desc respectivamene. 
    lista1_desc = mensaje_cifrado[0:int(len(mensaje_cifrado)/2)]
    lista2_desc = mensaje_cifrado[int(len(mensaje_cifrado)/2):len(mensaje_cifrado)]
    
    C_desc = linealMatriz(lista1_desc)
    M_desc = linealMatriz(lista2_desc)
    
    # 2. Calculamos la inversa de la clave K
    K_i = np.linalg.inv(K).tolist()

    # 3. Obtenemos la matriz P a partir de la matriz C descifrada y la 
    # inversa de la clave K  
    P_desc = np.dot(C_desc,K_i).tolist()

    # 4. Calculamos la inversa de la matriz M
    M_i = np.linalg.inv(M_desc).tolist()

    # 5. Calculamos la matriz T a partir de la inversa de M
    T_desc = np.round(np.dot(M_i,P_desc)).tolist()

    # 6. Representando el árbol generador minimal T, y usando 
    # la tabla de codificación obtenemos el mensaje cifrado.
    mensaje_desc = descifrarMatriz(T_desc,carac)

    return mensaje_desc


## Ejemplo práctico

Supongamos el escenario en el que Alicia y Bruno quieren realizar un intercambio
de información de manera confidencial. En concreto, Alicia quiere enviarle un mensaje a
Bruno de forma que nadie más pueda leerlo. Para ello, Alicia y Bruno cuentan con una clave
compartida que solo ellos conocen.

Denotemos dicha clave por K, y definámosla de la forma:

In [20]:
K = [[1, 1, 1, 1, 1, 1],
     [0, 1, 0, 0, 0, 1],
     [0, 0, 1, 0, 0, 1],
     [0, 0, 0, 1, 0, 1],
     [0, 0, 0, 0, 1, 1],
     [0, 0, 0, 0, 0, 1]]
K

[[1, 1, 1, 1, 1, 1],
 [0, 1, 0, 0, 0, 1],
 [0, 0, 1, 0, 0, 1],
 [0, 0, 0, 1, 0, 1],
 [0, 0, 0, 0, 1, 1],
 [0, 0, 0, 0, 0, 1]]

El mensaje que Alicia quiere enviarle a Bruno es el siguiente: *CLAVE*

In [21]:
mensaje_secreto = 'CLAVE'

En primer lugar, es necesario seleccionar un carácter especial antes del primer carácter para señalar por donde comenzar. Tomemos el carácter $S$.

In [22]:
carac = 'S'

Después, lo que debe realizar Alicia es convertir el mensaje a cifrar en vértices de un grafo. El conjunto de vértices va a estar formado por las cinco letras del mensaje *CLAVE* y, por cada par de caracteres secuenciales en el texto a cifrar, se añadirá una arista entre ambos vértices. Además, se unirá el primer y último vértices para formar un ciclo. 

Para poder asignarle un peso a cada arista, Alicia deberá de hacer uso de una tabla de codificación. En este ejemplo se usará la del diccionario *codificacion* definido previamente. El peso de cada arista corresponde con la distancia entre los caracteres que representan los vértices en la tabla de codificación. Por ejemplo, el peso de la arista que une los vértices $C$ y $L$ se corresponde con la distancia entre los valores $C$ y $L$ en la tabla de codificación.

In [23]:
codificarLetras('L','C')

9

A continuación, Alicia debe continuar añadiéndole aristas hasta conseguir un grafo completo, a las que asignará su correspondiente peso. En este caso, cada una de las aristas irá tomando un valor secuencial a partir del máximo peso de la tabla de codificación. Obteniendo así un grafo completo de orden 5.

Todos estos pasos para construir el grafo formado por caracteres, los realizamos aplicando el algoritmo **grafoMesaje** a partir del mensaje secreto y el carácter de inicio. Obteniendo así dicho grafo con su respectiva matriz de adyacencia $M$, definida de la forma:


In [24]:
g = grafoMensaje(mensaje_secreto, carac)
M = g.matrix
M

[[0, -16, 0, 0, 0, 0],
 [-16, 0, 9, 27, 28, -2],
 [0, 9, 0, -11, 29, 30],
 [0, 27, -11, 0, 21, 31],
 [0, 28, 29, 21, 0, -17],
 [0, -2, 30, 31, -17, 0]]

Ahora debemos encontrar el árbol generador minimal. Para ello haremos uso del *Algoritmo de Prim*, definido dentro de la clase **grafo**.

La matriz de adyacencia del Árbol Generador Minimal es:

In [25]:
T = g.primMST()
T

[[0, -16, 0, 0, 0, 0],
 [-16, 0, 9, 0, 0, -2],
 [0, 9, 0, -11, 0, 0],
 [0, 0, -11, 0, 0, 0],
 [0, 0, 0, 0, 0, -17],
 [0, -2, 0, 0, -17, 0]]

Ahora vamos a sustituir los ceros de la diagonal principal por el orden de los caracteres.

In [26]:
sustituirDiag(T)
T

[[0, -16, 0, 0, 0, 0],
 [-16, 1, 9, 0, 0, -2],
 [0, 9, 2, -11, 0, 0],
 [0, 0, -11, 3, 0, 0],
 [0, 0, 0, 0, 4, -17],
 [0, -2, 0, 0, -17, 5]]

Multiplicamos la matriz de adyacencia del grafo, M, y la del árbol generador minimal, T, para obtener la matriz P.

In [27]:
P = np.dot(M,T).tolist()
P

[[256, -16, -144, 0, 0, 32],
 [0, 341, -279, -18, 146, -486],
 [-144, -51, 202, -33, -394, -361],
 [-432, -134, 221, 121, -443, -256],
 [-448, 323, 79, -256, 289, -141],
 [32, 268, -299, -237, -68, 293]]

Ahora usamos la clave K para cifrar P

In [28]:
C = np.dot(P,K).tolist()
C

[[256, 240, 112, 256, 256, 128],
 [0, 341, -279, -18, 146, -296],
 [-144, -195, 58, -177, -538, -781],
 [-432, -566, -211, -311, -875, -923],
 [-448, -125, -369, -704, -159, -154],
 [32, 300, -267, -205, -36, -11]]

Lo que Alicia debe enviar a Bruno es C+M en una misma línea

In [29]:
lista1 = matrizLineal(C)
lista2 = matrizLineal(M)
mensaje_cifrado = lista1 + lista2

Ahora Bruno recibe el mensaje cifrado. Lo divide en dos listas y lo transforma en dos matrices, *C_desc* y *M_desc* respectivamente. 

In [30]:
lista1_desc = mensaje_cifrado[0:int(len(mensaje_cifrado)/2)]
lista2_desc = mensaje_cifrado[int(len(mensaje_cifrado)/2):len(mensaje_cifrado)]

In [31]:
C_desc = linealMatriz(lista1_desc)
M_desc = linealMatriz(lista2_desc)


Comprobamos que coinciden con las matrices *C* y *M*

In [32]:
C_desc == C and M_desc == M

True

A partir de la matriz C descifrada y la inversa de la clave K puede obtener la matriz P.

In [33]:
# Calculamos la inversa de K
K_i = np.linalg.inv(K).tolist()

In [34]:
# Multiplicamos la inversa de K por la matriz cifrada
P_desc = np.dot(C_desc,K_i).tolist()
P_desc

[[256.0, -16.0, -144.0, 0.0, 0.0, 32.0],
 [0.0, 341.0, -279.0, -18.0, 146.0, -486.0],
 [-144.0, -51.0, 202.0, -33.0, -394.0, -361.0],
 [-432.0, -134.0, 221.0, 121.0, -443.0, -256.0],
 [-448.0, 323.0, 79.0, -256.0, 289.0, -141.0],
 [32.0, 268.0, -299.0, -237.0, -68.0, 293.0]]

Comprobamos que efectivamente hemos descifrado bien la matriz P

In [35]:
P_desc == P

True

Bruno calcula la matriz T a partir de la inversa de M


In [36]:
# Calculamos la inversa de M
M_i = np.linalg.inv(M_desc).tolist()

In [37]:
# Multiplicamos la inversa de M por la matriz cifrada y 
# redondeamos por los decimales de calcular la inversa
T_desc = np.round(np.dot(M_i,P_desc)).tolist()
T_desc

[[-0.0, -16.0, 0.0, -0.0, 0.0, 0.0],
 [-16.0, 1.0, 9.0, 0.0, -0.0, -2.0],
 [0.0, 9.0, 2.0, -11.0, 0.0, 0.0],
 [0.0, 0.0, -11.0, 3.0, -0.0, 0.0],
 [-0.0, 0.0, 0.0, -0.0, 4.0, -17.0],
 [0.0, -2.0, 0.0, 0.0, -17.0, 5.0]]

Comprobamos que coincide con la matriz de adyacencia de árbol T.

In [38]:
T_desc == T

True

Representando el árbol generador minimal T, y usando la tabla de codificación Bruno obtiene el mensaje descifrado.

In [39]:
mensaje_desc = descifrarMatriz(T,carac)
mensaje_desc

['C', 'L', 'A', 'V', 'E']

A continuación, vamos a usar los algoritmos de cifrado y descifrado implementados.

In [40]:
print(codificar(mensaje_secreto,carac,K))

[256, 240, 112, 256, 256, 128, 0, 341, -279, -18, 146, -296, -144, -195, 58, -177, -538, -781, -432, -566, -211, -311, -875, -923, -448, -125, -369, -704, -159, -154, 32, 300, -267, -205, -36, -11, 0, -16, 0, 0, 0, 0, -16, 0, 9, 27, 28, -2, 0, 9, 0, -11, 29, 30, 0, 27, -11, 0, 21, 31, 0, 28, 29, 21, 0, -17, 0, -2, 30, 31, -17, 0]


In [41]:
print(decodificar(mensaje_cifrado,carac,K))

['C', 'L', 'A', 'V', 'E']


### Otro ejemplo

En este caso, partimos de la clave compartida $K$:

In [42]:
K = [[1, 1, 1, 1, 1, 1, 1, 1],
     [0, 1, 1, 1, 1, 1, 1, 1],
     [0, 0, 1, 1, 1, 1, 1, 1],
     [0, 0, 0, 1, 1, 1, 1, 1],
     [0, 0, 0, 0, 1, 1, 1, 1],     
     [0, 0, 0, 0, 0, 1, 1, 1],
     [0, 0, 0, 0, 0, 0, 1, 1],
     [0, 0, 0, 0, 0, 0, 0, 1]]
K

[[1, 1, 1, 1, 1, 1, 1, 1],
 [0, 1, 1, 1, 1, 1, 1, 1],
 [0, 0, 1, 1, 1, 1, 1, 1],
 [0, 0, 0, 1, 1, 1, 1, 1],
 [0, 0, 0, 0, 1, 1, 1, 1],
 [0, 0, 0, 0, 0, 1, 1, 1],
 [0, 0, 0, 0, 0, 0, 1, 1],
 [0, 0, 0, 0, 0, 0, 0, 1]]

El mensaje que Alicia quiere enviarle a Bruno es el siguiente: *CARLOTA*

In [43]:
mensaje_secreto = 'CARLOTA'

Primero se ha de seleccionar un carácter especial para señalar por donde empieza el mensaje. Tomemos el carácter $Y$.

In [44]:
carac = 'Y'

Después, Alicia convertirá el mensaje a cifrar en vértices de un grafo, en este caso formado por las SIETE letras del mensaje *CARLOTA*. Por cada par de caracteres secuenciales en el mensaje, se añadirá una arista entre ambos vértices. Además, será necesario unir el primer y último vértices para formar un ciclo. 

A la hora de asignar un peso a cada arista, Alicia usará una tabla de codificación. En este ejemplo se hará uso de la del diccionario *codificacion* definido previamente. El peso de cada arista corresponde con la distancia entre los caracteres que representan los vértices en la tabla de codificación. Por ejemplo, el peso de la arista que une los vértices $C$ y $A$ se corresponde con la distancia entre los valores $C$ y $A$ en la tabla de codificación.

In [45]:
codificarLetras('A','C')

-2

Alicia debe seguiar añadiendo aristas hasta obtener un grafo completo. A dichas aristas les asignará su peso correspondiente, que se trata de un valor secuencial a partir del máximo peso de la tabla de codificación. De esta forma, se obtiene un grafo completo de orden 7.

Se aplica el algoritmo **grafoMesaje** que engloba los pasos comentados anteriomenete. De él, obtenemos dicho grafo con su respectiva matriz de adyacencia $M$, definida de la forma:


In [46]:
g = grafoMensaje(mensaje_secreto, carac)
M = g.matrix
M

[[0, -22, 0, 0, 0, 0, 0, 0],
 [-22, 0, -2, 27, 28, 29, 30, 2],
 [0, -2, 0, 17, 31, 32, 33, 34],
 [0, 27, 17, 0, -6, 35, 36, 37],
 [0, 28, 31, -6, 0, 3, 38, 39],
 [0, 29, 32, 35, 3, 0, 5, 40],
 [0, 30, 33, 36, 38, 5, 0, -19],
 [0, 2, 34, 37, 39, 40, -19, 0]]

Ahora debemos encontrar el árbol generador minimal. Para ello haremos uso del *Algoritmo de Prim*, implementado como método de la clase **grafo**.

La matriz de adyacencia del Árbol Generador Minimal es:

In [47]:
T = g.primMST()
T

[[0, -22, 0, 0, 0, 0, 0, 0],
 [-22, 0, -2, 0, 0, 0, 0, 2],
 [0, -2, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, -6, 0, 0, 0],
 [0, 0, 0, -6, 0, 3, 0, 0],
 [0, 0, 0, 0, 3, 0, 5, 0],
 [0, 0, 0, 0, 0, 5, 0, -19],
 [0, 2, 0, 0, 0, 0, -19, 0]]

Se sustituyen los ceros de la diagonal principal por el orden de los caracteres.

In [48]:
sustituirDiag(T)
T

[[0, -22, 0, 0, 0, 0, 0, 0],
 [-22, 1, -2, 0, 0, 0, 0, 2],
 [0, -2, 2, 0, 0, 0, 0, 0],
 [0, 0, 0, 3, -6, 0, 0, 0],
 [0, 0, 0, -6, 4, 3, 0, 0],
 [0, 0, 0, 0, 3, 5, 5, 0],
 [0, 0, 0, 0, 0, 5, 6, -19],
 [0, 2, 0, 0, 0, 0, -19, 7]]

Multiplicamos la matriz de adyacencia del grafo, M, y la del árbol generador minimal, T, para obtener la matriz P.

In [49]:
P = np.dot(M,T).tolist()
P

[[484, -22, 44, 0, 0, 0, 0, -44],
 [0, 492, -4, -87, 37, 379, 287, -556],
 [44, 66, 4, -135, 118, 418, -288, -393],
 [-594, 67, -20, 36, 81, 337, -312, -371],
 [-616, 44, 6, -18, 45, 205, -498, -393],
 [-638, 45, 6, 87, -198, 34, -730, 243],
 [-660, -74, 6, -120, -49, 139, 386, -73],
 [-44, -66, 64, -123, 54, 222, 86, 365]]

Ahora usamos la clave K para cifrar P

In [50]:
C = np.dot(P,K).tolist()
C

[[484, 462, 506, 506, 506, 506, 506, 462],
 [0, 492, 488, 401, 438, 817, 1104, 548],
 [44, 110, 114, -21, 97, 515, 227, -166],
 [-594, -527, -547, -511, -430, -93, -405, -776],
 [-616, -572, -566, -584, -539, -334, -832, -1225],
 [-638, -593, -587, -500, -698, -664, -1394, -1151],
 [-660, -734, -728, -848, -897, -758, -372, -445],
 [-44, -110, -46, -169, -115, 107, 193, 558]]

Alicia debe enviar C+M en una misma línea

In [51]:
lista1 = matrizLineal(C)
lista2 = matrizLineal(M)
mensaje_cifrado = lista1 + lista2

Ahora Bruno recibe el mensaje cifrado. Lo divide en dos listas y lo transforma en dos matrices, *C_desc* y *M_desc*. 

In [52]:
lista1_desc = mensaje_cifrado[0:int(len(mensaje_cifrado)/2)]
lista2_desc = mensaje_cifrado[int(len(mensaje_cifrado)/2):len(mensaje_cifrado)]

In [53]:
C_desc = linealMatriz(lista1_desc)
M_desc = linealMatriz(lista2_desc)


Comprobamos que coinciden con las matrices *C* y *M*

In [54]:
C_desc == C and M_desc == M

True

A partir de la matriz C descifrada y la inversa de la clave K puede obtener la matriz P.

In [55]:
# Calculamos la inversa de K
K_i = np.linalg.inv(K).tolist()

In [56]:
# Multiplicamos la inversa de K por la matriz cifrada
P_desc = np.dot(C_desc,K_i).tolist()
P_desc

[[484.0, -22.0, 44.0, 0.0, 0.0, 0.0, 0.0, -44.0],
 [0.0, 492.0, -4.0, -87.0, 37.0, 379.0, 287.0, -556.0],
 [44.0, 66.0, 4.0, -135.0, 118.0, 418.0, -288.0, -393.0],
 [-594.0, 67.0, -20.0, 36.0, 81.0, 337.0, -312.0, -371.0],
 [-616.0, 44.0, 6.0, -18.0, 45.0, 205.0, -498.0, -393.0],
 [-638.0, 45.0, 6.0, 87.0, -198.0, 34.0, -730.0, 243.0],
 [-660.0, -74.0, 6.0, -120.0, -49.0, 139.0, 386.0, -73.0],
 [-44.0, -66.0, 64.0, -123.0, 54.0, 222.0, 86.0, 365.0]]

Verificamos que hemos descifrado bien la matriz P

In [57]:
P_desc == P

True

Bruno calcula la matriz T a partir de la inversa de M


In [58]:
# Calculamos la inversa de M
M_i = np.linalg.inv(M_desc).tolist()


In [59]:
# Multiplicamos la inversa de M por la matriz cifrada y 
# redondeamos por los decimales de calcular la inversa
T_desc = np.round(np.dot(M_i,P_desc)).tolist()
T_desc

[[0.0, -22.0, 0.0, -0.0, 0.0, 0.0, -0.0, 0.0],
 [-22.0, 1.0, -2.0, 0.0, -0.0, 0.0, -0.0, 2.0],
 [0.0, -2.0, 2.0, -0.0, 0.0, -0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 3.0, -6.0, 0.0, -0.0, -0.0],
 [-0.0, 0.0, -0.0, -6.0, 4.0, 3.0, 0.0, 0.0],
 [-0.0, 0.0, 0.0, -0.0, 3.0, 5.0, 5.0, -0.0],
 [0.0, 0.0, 0.0, -0.0, 0.0, 5.0, 6.0, -19.0],
 [-0.0, 2.0, -0.0, 0.0, 0.0, -0.0, -19.0, 7.0]]

Comprobamos que coincide con la matriz de adyacencia de árbol T.

In [60]:
T_desc == T

True

Representando el árbol generador minimal T, y haciendo uso de la tabla de codificación Bruno obtiene el mensaje descifrado.

In [61]:
mensaje_desc = descifrarMatriz(T,carac)
mensaje_desc

['C', 'A', 'R', 'L', 'O', 'T', 'A']

A continuación, usaremos los algoritmos de cifrado y descifrado implementados.

In [62]:
print(codificar(mensaje_secreto,carac,K))

[484, 462, 506, 506, 506, 506, 506, 462, 0, 492, 488, 401, 438, 817, 1104, 548, 44, 110, 114, -21, 97, 515, 227, -166, -594, -527, -547, -511, -430, -93, -405, -776, -616, -572, -566, -584, -539, -334, -832, -1225, -638, -593, -587, -500, -698, -664, -1394, -1151, -660, -734, -728, -848, -897, -758, -372, -445, -44, -110, -46, -169, -115, 107, 193, 558, 0, -22, 0, 0, 0, 0, 0, 0, -22, 0, -2, 27, 28, 29, 30, 2, 0, -2, 0, 17, 31, 32, 33, 34, 0, 27, 17, 0, -6, 35, 36, 37, 0, 28, 31, -6, 0, 3, 38, 39, 0, 29, 32, 35, 3, 0, 5, 40, 0, 30, 33, 36, 38, 5, 0, -19, 0, 2, 34, 37, 39, 40, -19, 0]


In [63]:
print(decodificar(mensaje_cifrado,carac,K))

['C', 'A', 'R', 'L', 'O', 'T', 'A']


## Resultados experimentales

A continuación, se probará el algoritmo implementado mediante una serie de ejecuciones para calcular los tiempos medios en milisegundos.

In [68]:
import string
import random

In [69]:
# Función para generar una clave compartida de tamaño n por n
# Dicha clave será una matriz triangular superior con sus
# elementos no nulos igual 1
def generarK(n):
    
    K = [[0 for column in range(n)] for row in range(n)]
    
    # Rellenamos la matriz
    for i in range(n):
        for j in range(i,n):
            K[i][j] = 1
    
    return K

In [70]:
# Función para generar un mensaje aleatorio de longitud n
def generarMensaje(n):
    return  ''.join(random.choice(string.ascii_uppercase) for _ in range(n))

In [72]:
# Número de mensajes distintos a cifrar
n = 50
# Número de iteraciones por cada mensaje
it = 30

# Tiempos del algoritmo de cifrado y descifrado
tiempos_cifrar = [None] * (n-3)
tiempos_descifrar = [None] * (n-3)

# Suma parcial de los tiempos de cifrado y descifrado
suma_cifrar = 0
suma_descifrar = 0

# Generamos mensajes de tamaño i a partir de 3 hasta n
# No podemos tomar el tamaño 1 ni 2 para
for i in range(1,n):
    # Generamos el mensaje y la clave compartida
    mensaje = generarMensaje(i)
    K = generarK(i+1)
    
    # Contabilizamos el tiempo empleado para cifrar
    inicio = time.time()
    mensaje_cifrado = codificar(mensaje,carac,K)
    fin = time.time()

    # Si el mensaje se ha podido cifrar, es decir,
    # si la matriz M es invertible
    if mensaje_cifrado != None:  

        # Realizamos it iteraciones
        for j in range(it):

            # Contabilizamos el tiempo empleado para cifrar
            inicio = time.time()
            codificar(mensaje,carac,K)
            fin = time.time()

            # Calculamos el tiempo empleado en cifrar
            suma_cifrar = suma_cifrar + (fin-inicio)*1000

            # Contabilizamos el tiempo empleado para descifrar
            inicio = time.time()
            decodificar(mensaje_cifrado,carac,K)
            fin = time.time()

            # Calculamos el tiempo empleado en cifrar            
            suma_descifrar = suma_descifrar + (fin-inicio)*1000

        tiempos_cifrar[i-3] = suma_cifrar / it
        tiempos_descifrar[i-3] = suma_descifrar / it
    
    # Reestablecemos el valor de la suma
    suma = 0
    
print("Tiempos de cifrado: ",tiempos_cifrar)
print("Tiempos de descifrado: ",tiempos_descifrar)

Tiempos de cifrado:  [0.27892589569091797, 0.3590504328409831, 0.4530191421508789, 0.5685091018676758, 0.7009108861287435, 0.9063561757405599, 1.1066436767578125, 1.3056596120198567, 1.527682940165202, 1.7914454142252605, 2.0676056543986, 2.3849169413248696, 2.7291138966878257, 3.1192938486735025, 3.538616498311361, 4.1083017985026045, 4.755441347757976, 5.407381057739258, 6.167030334472656, 6.9592634836832685, None, 7.89186954498291, 8.906364440917969, 9.752043088277182, 10.672799746195475, 11.635104815165201, 12.654868761698404, 13.879696528116861, 15.18255869547526, 16.671864191691082, 18.31834316253662, 19.68204975128174, 21.14435036977132, 22.728689511617024, 24.329678217569988, 25.998791058858234, None, 27.822359402974445, 29.768983523050945, 31.89397652943929, 34.053985277811684, 36.29759152730306, 38.61611684163412, 41.05098247528076, 43.53416760762533, 46.1771567662557, 48.9920457204183]
Tiempos de descifrado:  [0.29006004333496094, 0.38983821868896484, 0.5013783772786459, 0.6